In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

# Boundary Value Problems

## Example: 1-D Reaction-diffusion equation

The one-dimensional, time-dependent reaction-diffusion equation can be written as:
    
$$ \frac{\partial c}{\partial t} = \mathscr{D} \frac{\partial^2 c}{\partial x^2} + R(c) $$

where $\mathscr{D}$ is the diffusion constant and $R(c)$ is the rate of generation by chemical reaction.

We are interested in the steady-state concentration profile given a first order reaction and the following boundary conditions:

$$ R(c) = -k c $$
$$ c(0) = C_0, c(L) = C_0 $$
$$ 0 < x < L $$

We re-write the reaction-diffusion equation as:

$$ \frac{d^2u}{d\xi^2} = Da*u $$

Where we have introduce the following dimensionless variables:

$$ u = \frac{c}{C_0} $$
$$ \xi = \frac{x}{L} $$
$$ Da = \frac{k C_0^{n-1}}{\mathscr{D}L^2} $$

$Da$ is known as the Damköhler number, which describes the relative ratio between the chemical reaction rate (of order $n$) and mass transfer rate.

Finally, we re-write this as a system of coupled, first-order ODEs:

$$ \frac{du}{d\xi} = v $$
$$ \frac{dv}{d\xi} = Da*u $$
$$ u(0) = 1, u(1) = 1 $$
$$ 0 < x < 1 $$

In [ ]:
# Let's solve the equation for a few different values of Da
for Da in [1e-2, 1e-1, 1., 10.]:

    def f(x, y):
        return np.stack((y[1], y[0]*Da), axis=0)

    def bc(ya, yb):
        return [ya[0]-1, yb[0]-1]

    x = np.linspace(0, 1)
    u = np.zeros((2, x.size))
    sol = solve_bvp(f, bc, x, u)
    u_sol = sol.y[0]
    plt.scatter(x, u_sol, label="Da: {}".format(Da))

plt.xlabel("$\\xi=x/L$")
plt.ylabel("$u=c/C_0$")
plt.legend()
plt.show()
plt.close()

## Exercise: 1-D Reaction-diffusion equation

Try repeating the above analysis for a second-order reaction:
    
$$ R(c) = k c^2 $$

# Gradients

Numpy has a built-in function for performing numerical differentiation. The gradient is computed using a second-order, central difference method on the interior points and a first- or second-order forward/backward difference method on either boundary. Let's use the `np.gradient` function to check our answers above.

We wrote down the reaction-diffusion equation as:

$$ \frac{d^2u}{d\xi^2} = Da*u $$

Does $ \frac{d}{dx} \frac{d}{dx} u = Da*u $?

In [ ]:
u_sol_grad = np.gradient(u_sol, x, edge_order=2)
u_sol_grad_grad = np.gradient(u_sol_grad, x, edge_order=2)
plt.plot(x, Da*u_sol)
plt.plot(x, u_sol_grad_grad)

# Partial differential equations

We can combine `np.gradients` with `scipy.integrate.solve_ivp` to solve partial differential equations with independent time and space variables. Let's re-visit the problem of 1-D heat conduction, this time considering the time-dependent solution.

## Example: 1-D Heat Conduction

$$ \frac{\partial u}{\partial t} = \alpha \frac{\partial^2u}{\partial x^2} $$
$$ u(0, t) = T_0, u(1, t) = T_1, u(x, 0) = T_0 $$
$$ 0<x<1, t>0 $$

In [ ]:
from scipy.integrate import solve_ivp

def f(t, y, x, alpha):
    rhs = alpha * np.gradient(np.gradient(y, x), x)
    # setting the RHS on the boundary to 0 preserves the boundary conditions
    rhs[0] = 0.
    rhs[-1] = 0.
    return rhs
    
alpha = 0.1
T0 = 100.
T1 = 275.
t_span = [0., 5.]
y0 = np.ones((50,)) * T0
y0[-1] = T1
x = np.linspace(0, 1, 50)
t_eval = np.linspace(*t_span, 100)
sol = solve_ivp(f, t_span, y0, t_eval=t_eval, args=(x, alpha))

In [ ]:
y_plt = sol.y[:, ::10]
for i in range(y_plt.shape[-1]):
    plt.plot(x, y_plt)
plt.legend(['t={}'.format(i*5/100*10) for i in range(y_plt.shape[-1])])
plt.show()
plt.close()

## Converting Plots to Video

Here's a nice routine for creating an animation of the transient heat conduction. The idea is to construct a function `animate` that creates the video one frame at a time, and the pass that function to `matplotlib.animation.FuncAnimation`.

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML
fig = plt.figure()
def animate(i):
    plt.clf()
    plt.xlabel("x")
    plt.ylabel("T [K]")
    p = plt.plot(x, sol.y[:, i])
    return p
ani = animation.FuncAnimation(fig, animate, frames=len(sol.y))
HTML(ani.to_html5_video())

## Exercise: 1-D Reaction-Diffusion

The one-dimensional, time-dependent reaction-diffusion equation can be written as:
    
$$ \frac{\partial u}{\partial t} = \frac{\partial^2 u}{\partial \xi^2} - Da * u $$

$$ u(0) = 1, u(1) = 1 $$
$$ 0 < \xi < 1, t > 0 $$

where we use the same dimensionless variables defined above. Solve for the dimensionless concentration, u, when Da=10.

